<a href="https://colab.research.google.com/github/EricMiao01/Scheduling_Problem/blob/main/%E6%95%B8%E9%87%8F%E7%A0%94%E7%A9%B6%E6%96%B9%E6%B3%95_%E5%A4%9A%E7%9B%AE%E6%A8%99%E8%A6%8F%E5%8A%83%E5%AF%A6%E4%BD%9C.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

## **取得員工可上班日調查表**

In [293]:
import pandas as pd
import numpy as np

In [294]:
def highlight_availability(val):
    if val == 1:
        return 'background-color: gray'  # 可以上班的底色為灰色
    else:
        return ''

In [295]:
seed_value = 5201314  # 設定種子值
np.random.seed(seed_value)  # 固定隨機數生成器的種子
data = np.random.randint(0, 2, size = (15, 10)) # 生成15位員工在10個工作天中的可出席日

In [296]:
df = pd.DataFrame(data, columns = [str(i) for i in range(1, 11)])
df.columns.name = 'WorkDay'
df.index.name = 'Employee'
print("員工可上班時間調查表:")
df.style.applymap(highlight_availability)

員工可上班時間調查表:


WorkDay,1,2,3,4,5,6,7,8,9,10
Employee,,,,,,,,,,
0,0,0,1,0,0,0,0,0,0,1
1,0,1,1,0,0,0,0,0,0,0
2,1,0,0,0,0,1,1,0,0,1
3,0,1,0,0,1,1,0,0,1,1
4,1,1,0,1,1,0,0,0,1,1
5,0,1,1,1,0,0,1,0,0,0
6,0,1,1,1,0,0,0,0,0,0
7,0,1,1,1,0,0,1,1,1,0
8,1,0,1,1,1,1,0,0,0,0


In [297]:
# 輸出調查表
df.to_csv('員工可上班時間表.csv')

## **設定公司每個上班日所需人數**

In [298]:
# 公司每個上班日需要的員工數量
workdayRequirements = {
    '1': 3,
    '2': 4,
    '3': 3,
    '4': 3,
    '5': 4,
    '6': 2,
    '7': 3,
    '8': 3,
    '9': 3,
    '10': 2
}

## **設定每日人事成本**

In [299]:
daily_wage = 1600 # 8hr * 180NTW

---

# **1. 單一目標: Goal1 讓員工上班日盡量平均**

上班最多天員工的上班天數 - 上班最少天員工的上班天數 <= 1

## **建立模型**

In [300]:
try:
    from gurobipy import *
except:
    !pip install gurobipy
finally:
    from gurobipy import *

In [301]:
# 建模
m1 = Model('員工排班')

### **設計變數**

In [302]:
# Decision Variable
x = m1.addVars(df.index, df.columns, vtype = GRB.BINARY, name = 'x')


for employee in df.index:
    for workday in df.columns:
        if df.loc[employee, workday] != 1:
            x[employee, workday].UB = 0  # 設置不可上班的變數上限為 0

### **輔助變數**

In [303]:
# Auxiliary Variable
totalWorkdays = m1.addVars(df.sum(axis=1).index, name="TotalWorkdays")
minWorkday = m1.addVar(name='minWorkday')
maxWorkday = m1.addVar(name='maxWorkday')
n1 = m1.addVar(lb = 0, name='n1')
p1 = m1.addVar(lb = 0, name='p1')

In [304]:
# 限制式: 每個工作日都需要有一定數量的員工上班
workday_requirement = m1.addConstrs((x.sum('*', w) >= workdayRequirements[w] for w in df.columns),
                                   name = 'workdayRequirement')

# 限制式: 每個員工可出席的工作日總數
num_workdays = m1.addConstrs((x.sum(e, '*') == totalWorkdays[e] for e in df.index), name='totalWorkdays')

min_workday = m1.addGenConstrMin(minWorkday, totalWorkdays, name = 'minWorkdays')
max_workday = m1.addGenConstrMax(maxWorkday, totalWorkdays, name = 'maxWorkdays')

goal1 = m1.addConstr(((maxWorkday - minWorkday) + n1 - p1 == 1), name='goal1')


In [305]:
# 加入目標式
m1.setObjective(n1 + 10*p1, GRB.MINIMIZE)

m1.optimize()

Gurobi Optimizer version 10.0.1 build v10.0.1rc0 (linux64)

CPU model: AMD EPYC 7B12, instruction set [SSE2|AVX|AVX2]
Thread count: 1 physical cores, 2 logical processors, using up to 2 threads

Optimize a model with 26 rows, 169 columns and 319 nonzeros
Model fingerprint: 0xa7595b67
Model has 2 general constraints
Variable types: 19 continuous, 150 integer (150 binary)
Coefficient statistics:
  Matrix range     [1e+00, 1e+00]
  Objective range  [1e+00, 1e+01]
  Bounds range     [1e+00, 1e+00]
  RHS range        [1e+00, 4e+00]
Presolve added 55 rows and 0 columns
Presolve removed 0 rows and 41 columns
Presolve time: 0.00s
Presolved: 81 rows, 128 columns, 546 nonzeros
Variable types: 33 continuous, 95 integer (86 binary)
Found heuristic solution: objective 30.0000000
Found heuristic solution: objective 10.0000000
Found heuristic solution: objective 1.0000000
Found heuristic solution: objective 0.0000000

Root relaxation: cutoff, 0 iterations, 0.00 seconds (0.00 work units)

Explored 1 n

## **執行結果**

In [306]:
# 創建一個空的 DataFrame 來存放結果
result_df = pd.DataFrame(0, index=df.index, columns=df.columns)

# 將解中為 1 的變數對應的位置設置為 1
for employee, workday in x.keys():
    if x[employee, workday].X == 1:
        result_df.loc[employee, workday] = 1

# 結果
print("班表:")
result_df.style.applymap(highlight_availability)

班表:


WorkDay,1,2,3,4,5,6,7,8,9,10
Employee,,,,,,,,,,
0,0,0,1,0,0,0,0,0,0,1
1,0,1,1,0,0,0,0,0,0,0
2,1,0,0,0,0,1,1,0,0,0
3,0,1,0,0,1,1,0,0,0,0
4,1,1,0,0,1,0,0,0,0,0
5,0,1,1,1,0,0,0,0,0,0
6,0,0,1,1,0,0,0,0,0,0
7,0,0,0,1,0,0,1,0,1,0
8,1,0,0,0,1,0,0,0,0,0


In [307]:
print(f'工作天數全距: {result_df.sum(axis=1).max() - result_df.sum(axis=1).min()} 天')
print(f'人事成本{daily_wage * result_df.sum().sum()} 元')

工作天數全距: 1 天
人事成本56000 元


In [308]:
# 輸出班表
result_df.to_csv('考慮目標一班表.csv')

---

# **2. 單一目標: Goal2 成本極小化**

最小化人事成本

In [309]:
# 建模
m2 = Model('員工排班')

# Decision Variable
x = m2.addVars(df.index, df.columns, vtype = GRB.BINARY, name = 'x')


for employee in df.index:
    for workday in df.columns:
        if df.loc[employee, workday] != 1:
            x[employee, workday].UB = 0  # 設置不可上班的變數上限為 0

In [310]:
# 輔助變數
totalWorkdays = m2.addVars(df.sum(axis=1).index, name="TotalWorkdays")
n2 = m2.addVar(lb = 0, name = 'n2')
p2 = m2.addVar(lb = 0, name = 'p2')

# 限制式: 每個工作日都需要有一定數量的員工上班
workday_requirement = m2.addConstrs((x.sum('*', w) >= workdayRequirements[w] for w in df.columns),
                                   name = 'workdayRequirement')

# 限制式: 每個員工可出席的工作日總數
num_workdays = m2.addConstrs((x.sum(e, '*') == totalWorkdays[e] for e in df.index), name='totalWorkdays')

# 限制式: 目標函數
goal2 = m2.addConstr((daily_wage*totalWorkdays.sum() + n2 - p2 == 50000), name='goal2')

In [311]:
# 目標式
m2.setObjective(n2 + 10*p2, GRB.MINIMIZE)

m2.optimize()

Gurobi Optimizer version 10.0.1 build v10.0.1rc0 (linux64)

CPU model: AMD EPYC 7B12, instruction set [SSE2|AVX|AVX2]
Thread count: 1 physical cores, 2 logical processors, using up to 2 threads

Optimize a model with 26 rows, 167 columns and 332 nonzeros
Model fingerprint: 0x393012e4
Variable types: 17 continuous, 150 integer (150 binary)
Coefficient statistics:
  Matrix range     [1e+00, 2e+03]
  Objective range  [1e+00, 1e+01]
  Bounds range     [1e+00, 1e+00]
  RHS range        [2e+00, 5e+04]
Found heuristic solution: objective 2000.0000000
Presolve removed 25 rows and 165 columns
Presolve time: 0.00s
Presolved: 1 rows, 2 columns, 2 nonzeros
Found heuristic solution: objective 400.0000000
Variable types: 0 continuous, 2 integer (0 binary)

Explored 0 nodes (0 simplex iterations) in 0.01 seconds (0.00 work units)
Thread count was 2 (of 2 available processors)

Solution count 2: 400 2000 

Optimal solution found (tolerance 1.00e-04)
Best objective 4.000000000000e+02, best bound 4.0000

In [312]:
# 創建一個空的 DataFrame 來存放結果
result_df = pd.DataFrame(0, index=df.index, columns=df.columns)

# 將解中為 1 的變數對應的位置設置為 1
for employee, workday in x.keys():
    if x[employee, workday].X == 1:
        result_df.loc[employee, workday] = 1

# 結果
print("班表:")
result_df.style.applymap(highlight_availability)

班表:


WorkDay,1,2,3,4,5,6,7,8,9,10
Employee,,,,,,,,,,
0,0,0,0,0,0,0,0,0,0,0
1,0,1,1,0,0,0,0,0,0,0
2,0,0,0,0,0,1,0,0,0,0
3,0,1,0,0,1,0,0,0,0,1
4,0,0,0,0,1,0,0,0,0,0
5,0,1,0,1,0,0,0,0,0,0
6,0,0,0,0,0,0,0,0,0,0
7,0,1,1,1,0,0,1,0,1,0
8,1,0,0,1,1,0,0,0,0,0


In [313]:
print(f'工作天數全距: {result_df.sum(axis=1).max() - result_df.sum(axis=1).min()} 天')
print(f'人事成本{daily_wage * result_df.sum().sum()} 元')

工作天數全距: 5 天
人事成本49600 元


In [314]:
# 輸出班表
result_df.to_csv('考慮目標二班表.csv')

---

# **3. 多目標規劃: G1 + G2 同時考慮**

In [315]:
# 建模
m3 = Model('員工排班')

# Decision Variable
x = m3.addVars(df.index, df.columns, vtype = GRB.BINARY, name = 'x')

for employee in df.index:
    for workday in df.columns:
        if df.loc[employee, workday] != 1:
            x[employee, workday].UB = 0  # 設置不可上班的變數上限為 0
            

In [316]:
# Auxiliary Variable
totalWorkdays = m3.addVars(df.sum(axis=1).index, name="TotalWorkdays")
minWorkday = m3.addVar(name='minWorkday')
maxWorkday = m3.addVar(name='maxWorkday')
n1 = m3.addVar(lb = 0, name='n1')
p1 = m3.addVar(lb = 0, name='p1')
n2 = m3.addVar(lb = 0, name = 'n2')
p2 = m3.addVar(lb = 0, name = 'p2')

In [317]:
# 限制式: 每個工作日都需要有一定數量的員工上班
workday_requirement = m3.addConstrs((x.sum('*', w) >= workdayRequirements[w] for w in df.columns),
                                   name = 'workdayRequirement')

# 限制式: 每個員工可出席的工作日總數
num_workdays = m3.addConstrs((x.sum(e, '*') == totalWorkdays[e] for e in df.index), name='totalWorkdays')

min_workday = m3.addGenConstrMin(minWorkday, totalWorkdays, name = 'minWorkdays')
max_workday = m3.addGenConstrMax(maxWorkday, totalWorkdays, name = 'maxWorkdays')

# 限制式: 目標函數
goal1 = m3.addConstr(((maxWorkday - minWorkday) + n1 - p1 == 1), name='goal1')

# 限制式: 目標函數
goal2 = m3.addConstr((daily_wage*totalWorkdays.sum() + n2 - p2 == 50000), name='goal2')

In [318]:
# 目標式
m3.setObjective(0*n1 + 10*p1 + 0*n2 + 10*p2, GRB.MINIMIZE)

m3.optimize()

Gurobi Optimizer version 10.0.1 build v10.0.1rc0 (linux64)

CPU model: AMD EPYC 7B12, instruction set [SSE2|AVX|AVX2]
Thread count: 1 physical cores, 2 logical processors, using up to 2 threads

Optimize a model with 27 rows, 171 columns and 336 nonzeros
Model fingerprint: 0xd562b646
Model has 2 general constraints
Variable types: 21 continuous, 150 integer (150 binary)
Coefficient statistics:
  Matrix range     [1e+00, 2e+03]
  Objective range  [1e+01, 1e+01]
  Bounds range     [1e+00, 1e+00]
  RHS range        [1e+00, 5e+04]
Presolve added 27 rows and 0 columns
Presolve removed 0 rows and 99 columns
Presolve time: 0.00s
Presolved: 54 rows, 72 columns, 205 nonzeros
Variable types: 0 continuous, 72 integer (56 binary)
Found heuristic solution: objective 40.0000000

Root relaxation: objective 0.000000e+00, 52 iterations, 0.00 seconds (0.00 work units)

    Nodes    |    Current Node    |     Objective Bounds      |     Work
 Expl Unexpl |  Obj  Depth IntInf | Incumbent    BestBd   Gap |

In [319]:
# 創建一個空的 DataFrame 來存放結果
result_df = pd.DataFrame(0, index=df.index, columns=df.columns)

# 將解中為 1 的變數對應的位置設置為 1
for employee, workday in x.keys():
    if x[employee, workday].X == 1:
        result_df.loc[employee, workday] = 1

# 結果
print("班表:")
result_df.style.applymap(highlight_availability)

班表:


WorkDay,1,2,3,4,5,6,7,8,9,10
Employee,,,,,,,,,,
0,0,0,1,0,0,0,0,0,0,1
1,0,1,1,0,0,0,0,0,0,0
2,1,0,0,0,0,0,1,0,0,0
3,0,0,0,0,1,0,0,0,1,0
4,1,0,0,0,1,0,0,0,0,0
5,0,1,1,0,0,0,0,0,0,0
6,0,1,0,1,0,0,0,0,0,0
7,0,1,0,0,0,0,0,0,1,0
8,0,0,0,0,1,1,0,0,0,0


In [320]:
print(f'工作天數全距: {result_df.sum(axis=1).max() - result_df.sum(axis=1).min()} 天')
print(f'人事成本: {daily_wage * result_df.sum().sum()} 元')

工作天數全距: 0 天
人事成本: 48000 元


In [321]:
# 輸出班表
result_df.to_csv('考慮目標一及目標二班表.csv')